In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import re
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
import os

2024-05-09 20:22:39.672746: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 20:22:39.722529: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 20:22:39.722563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 20:22:39.723678: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 20:22:39.731554: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# csv 파일을 dataframe으로 변환
df_outfit = pd.read_csv('../data/outfit(male)/outfit(male).csv')
df_weather = pd.read_csv('../data/2022-08-01_to_2024-04-30.csv', encoding='cp949')
# 필요한 columns만 추출
df_outfit = df_outfit[['userId', '상의', '아우터', '하의', '신발', '액세서리', '작성일']].copy()
df_temp = df_weather[['일시', '평균기온(°C)']].copy()

# '작성일'과 '일시' 열을 datetime 형식으로 변환
df_outfit['작성일'] = pd.to_datetime(df_outfit['작성일'], format='%Y년 %m월 %d일')
df_temp['일시'] = pd.to_datetime(df_temp['일시'])

# 두 dataframe을 날짜를 기준으로 병합
df_merged = pd.merge(df_outfit, df_temp, left_on='작성일', right_on='일시').drop('일시', axis=1)

df_merged

,userId,상의,아우터,하의,신발,액세서리,작성일,평균기온(°C)
0,1,"반팔 티, 셔츠/블라우스",재킷,반바지,구두/로퍼,NaN,2024-04-24,13.2
1,1,반팔 티,재킷,반바지,운동화,기타 모자,2024-04-19,17.6
2,1,반팔 티,재킷,반바지,구두/로퍼,장목양말,2024-04-15,16.0
3,1,반팔 티,NaN,나일론 팬츠,구두/로퍼,NaN,2024-04-09,15.3
4,1,반팔 티,집업,면바지,구두/로퍼,NaN,2024-04-05,14.0
...,...,...,...,...,...,...,...,...
1333,14,반팔 티,NaN,반바지,운동화,"기타 모자, 장목양말",2024-04-23,17.3
1334,14,반팔 티,NaN,카고바지,운동화,기타 모자,2024-04-24,13.2
1335,14,반팔 티,집업,나일론 팬츠,스니커즈/캔버스,기타 모자,2024-04-25,14.4
1336,14,"반팔 티, 셔츠/블라우스",NaN,반바지,구두/로퍼,장목양말,2024-04-26,17.8


In [3]:
# '상의', '아우터', '하의', '신발', '엑세서리' 열의 결측값을 '~ 없음'으로 대체
columns = ['상의', '아우터', '하의', '신발', '액세서리']
df_notnull = df_merged.copy()
for column in columns:
    df_notnull[column] = df_merged[column].fillna(column + ' 없음')

In [4]:
'''df_notnull[df_notnull['아우터'].str.contains('재킷 2')]'''

"df_notnull[df_notnull['아우터'].str.contains('재킷 2')]"

In [5]:
'''df_dup[df_dup['아우터'].str.contains('니트')]'''

"df_dup[df_dup['아우터'].str.contains('니트')]"

In [6]:
# 2가 붙은 단어를 두 번 반복하는 함수
def duplicate_word(text):
    words = text.split(', ')
    for i, word in enumerate(words):
        if '2' in word:
            words[i] = word.replace('2', '') + ', ' + word.replace('2', '')
    return ', '.join(words)

In [7]:
# 2가 붙은 단어를 두 번 반복한 dataframe df_dup 생성
df_dup = df_notnull.copy()
for column in columns:
    df_dup[columns] = df_notnull[columns].map(duplicate_word)

In [8]:
df_dup.columns

Index(['userId', '상의', '아우터', '하의', '신발', '액세서리', '작성일', '평균기온(°C)'], dtype='object')

In [9]:
# 옷의 조합 컬럼 생성 (상의, 아우터, 하의, 신발, 엑세서리의 각 값들을 하나의 문자열로 조합하여 하나의 컬럼으로 만듦)
df_combination = df_dup.copy()
df_combination['옷 조합'] = df_dup['상의'] + ', ' + df_dup['아우터'] + ', ' + df_dup['하의'] + ', ' + df_dup['신발'] + ', ' + df_dup['액세서리']
df_combination.drop(columns=['상의', '아우터', '하의', '신발', '액세서리'], inplace=True)

In [10]:
df_combination.columns

Index(['userId', '작성일', '평균기온(°C)', '옷 조합'], dtype='object')

In [11]:
'''df_combination[df_combination['옷 조합'].str.contains('니트 , 니트')]'''

"df_combination[df_combination['옷 조합'].str.contains('니트 , 니트')]"

In [12]:
'''# 옷의 조합 컬럼의 공백 제거
df_combination['옷 조합'] = df_combination['옷 조합'].str.replace(' ', '')'''

"# 옷의 조합 컬럼의 공백 제거\ndf_combination['옷 조합'] = df_combination['옷 조합'].str.replace(' ', '')"

In [13]:
# 쉼표를 기준으로 텍스트를 나누는 함수
def comma_tokenizer(s):
    return s.split(', ')

vectorizer = CountVectorizer(tokenizer=comma_tokenizer)

O = vectorizer.fit_transform(df_combination['옷 조합'])

# multi-hot encoding된 데이터를 numpy array로 변환
df_encoded = pd.DataFrame(O.toarray().tolist(), columns=vectorizer.get_feature_names_out())
npa = np.array(df_encoded)
npa.shape

/home/ksy/anaconda3/envs/ondoset/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1338, 45)

In [14]:
# 값이 2 이상인 행의 인덱스
rows_with_value_2 = df_encoded[(df_encoded >= 2).any(axis=1)]
rows_with_value_2.index

Index([86, 317, 430, 435, 560, 593, 633, 640, 793, 1039], dtype='int64')

In [15]:
# 값이 2 이상인 열의 이름을 찾습니다.
columns_with_value_over_2 = df_encoded.columns[(df_encoded >= 2).any()]

# 특정 행에 대해 이를 기록합니다.
record = df_encoded.loc[rows_with_value_2.index, columns_with_value_over_2]
record

,니트,민소매 티,반팔 티,재킷
86,0,2,1,1
317,2,0,0,0
430,0,0,2,0
435,0,0,2,0
560,0,0,0,2
593,0,0,2,0
633,1,0,2,1
640,0,0,1,2
793,2,0,0,0
1039,0,0,2,0


In [16]:
# 단어장 확인
vectorizer.get_feature_names_out()

array(['가디건', '가죽 바지', '구두/로퍼', '기타 모자', '긴팔 티', '나일론 팬츠', '니트', '데님팬츠',
       '레더부츠', '레인부츠', '마스크', '맨투맨', '머플러', '면바지', '민소매 티', '바람막이', '반바지',
       '반팔', '반팔 니트', '반팔 셔츠/블라우스', '반팔 티', '비니', '샌들/슬리퍼', '셔츠/블라우스',
       '스니커즈/캔버스', '스카프', '슬랙스', '아우터 없음', '액세서리 없음', '양말', '운동화', '장목양말',
       '재킷', '점퍼', '조끼', '집업', '카고바지', '코트', '털 모자', '트레이닝/조거 팬츠', '패딩',
       '패딩슈즈', '패딩조끼', '하의 없음', '후드티'], dtype=object)

In [17]:
# numpy array를 list로 변환 후 clothes_combination 컬럼에 대입
df_combination['옷 조합'] = npa.tolist()
df_combination['옷 조합']

0       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
                              ...                        
1333    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1334    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1335    [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1336    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1337    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
Name: 옷 조합, Length: 1338, dtype: object

In [18]:
# multi-hot encoding된 데이터를 다시 텍스트로 변환
df_combination['옷 조합'] = vectorizer.inverse_transform(npa)
df_combination['옷 조합'] 

0        [구두/로퍼, 반바지, 반팔 티, 셔츠/블라우스, 액세서리 없음, 재킷]
1                     [기타 모자, 반바지, 반팔 티, 운동화, 재킷]
2                    [구두/로퍼, 반바지, 반팔 티, 장목양말, 재킷]
3          [구두/로퍼, 나일론 팬츠, 반팔 티, 아우터 없음, 액세서리 없음]
4                 [구두/로퍼, 면바지, 반팔 티, 액세서리 없음, 집업]
                          ...                    
1333        [기타 모자, 반바지, 반팔 티, 아우터 없음, 운동화, 장목양말]
1334             [기타 모자, 반팔 티, 아우터 없음, 운동화, 카고바지]
1335          [기타 모자, 나일론 팬츠, 반팔 티, 스니커즈/캔버스, 집업]
1336    [구두/로퍼, 반바지, 반팔 티, 셔츠/블라우스, 아우터 없음, 장목양말]
1337             [기타 모자, 데님팬츠, 반팔 티, 아우터 없음, 운동화]
Name: 옷 조합, Length: 1338, dtype: object

In [19]:
# 하나의 문자열로 변환
df_combtest = df_combination.copy()
df_combtest['옷 조합'] = df_combination['옷 조합'].apply(lambda x: ', '.join(map(str, x)))

In [20]:
# multi-hot encoding의 값이 2 이상인 경우, 해당 단어를 두 번 반복
for i in record.index:
    old_value = df_combtest.loc[i, '옷 조합']
    for col in record.columns:
        if record.loc[i, col] >= 2:
            old_value = old_value.replace(col, col + ', ' + col)
    df_combtest.loc[i, '옷 조합'] = old_value

In [21]:
df_combtest.loc[rows_with_value_2.index, '옷 조합']

86      기타 모자, 데님팬츠, 민소매 티, 민소매 티, 반팔 티, 셔츠/블라우스, 스니커즈...
317               니트, 니트, 데님팬츠, 스니커즈/캔버스, 아우터 없음, 액세서리 없음
430           데님팬츠, 반팔 티, 반팔 티, 스니커즈/캔버스, 아우터 없음, 액세서리 없음
435              면바지, 반팔 티, 반팔 티, 샌들/슬리퍼, 아우터 없음, 액세서리 없음
560               구두/로퍼, 기타 모자, 데님팬츠, 셔츠/블라우스, 재킷, 재킷, 조끼
593                    반바지, 반팔 티, 반팔 티, 아우터 없음, 운동화, 장목양말
633               구두/로퍼, 니트, 면바지, 반팔 티, 반팔 티, 액세서리 없음, 재킷
640                     구두/로퍼, 면바지, 반팔 티, 액세서리 없음, 재킷, 재킷
793                      니트, 니트, 비니, 스니커즈/캔버스, 카고바지, 패딩조끼
1039              반바지, 반팔 티, 반팔 티, 스니커즈/캔버스, 아우터 없음, 장목양말
Name: 옷 조합, dtype: object

In [22]:
df_combtest

,userId,작성일,평균기온(°C),옷 조합
0,1,2024-04-24,13.2,"구두/로퍼, 반바지, 반팔 티, 셔츠/블라우스, 액세서리 없음, 재킷"
1,1,2024-04-19,17.6,"기타 모자, 반바지, 반팔 티, 운동화, 재킷"
2,1,2024-04-15,16.0,"구두/로퍼, 반바지, 반팔 티, 장목양말, 재킷"
3,1,2024-04-09,15.3,"구두/로퍼, 나일론 팬츠, 반팔 티, 아우터 없음, 액세서리 없음"
4,1,2024-04-05,14.0,"구두/로퍼, 면바지, 반팔 티, 액세서리 없음, 집업"
...,...,...,...,...
1333,14,2024-04-23,17.3,"기타 모자, 반바지, 반팔 티, 아우터 없음, 운동화, 장목양말"
1334,14,2024-04-24,13.2,"기타 모자, 반팔 티, 아우터 없음, 운동화, 카고바지"
1335,14,2024-04-25,14.4,"기타 모자, 나일론 팬츠, 반팔 티, 스니커즈/캔버스, 집업"
1336,14,2024-04-26,17.8,"구두/로퍼, 반바지, 반팔 티, 셔츠/블라우스, 아우터 없음, 장목양말"


In [23]:
# 평균기온(°C) column의 최대값과 최솟값
max_temp = df_combtest['평균기온(°C)'].max()
min_temp = df_combtest['평균기온(°C)'].min()
print(max_temp, min_temp)

30.9 -11.8


In [24]:
df_limit = df_combtest.copy()
# 평균기온(°C) column을 5도 간격으로 범주화하여 0, 1, 2, ...로 변환
df_limit['평균기온(°C)'] = pd.cut(df_limit['평균기온(°C)'],bins=np.round(np.arange(min_temp -5, max_temp+5, 5), 1), labels=np.arange(0, (max_temp-min_temp)//5+2))
df_limit

,userId,작성일,평균기온(°C),옷 조합
0,1,2024-04-24,5.0,"구두/로퍼, 반바지, 반팔 티, 셔츠/블라우스, 액세서리 없음, 재킷"
1,1,2024-04-19,6.0,"기타 모자, 반바지, 반팔 티, 운동화, 재킷"
2,1,2024-04-15,6.0,"구두/로퍼, 반바지, 반팔 티, 장목양말, 재킷"
3,1,2024-04-09,6.0,"구두/로퍼, 나일론 팬츠, 반팔 티, 아우터 없음, 액세서리 없음"
4,1,2024-04-05,6.0,"구두/로퍼, 면바지, 반팔 티, 액세서리 없음, 집업"
...,...,...,...,...
1333,14,2024-04-23,6.0,"기타 모자, 반바지, 반팔 티, 아우터 없음, 운동화, 장목양말"
1334,14,2024-04-24,5.0,"기타 모자, 반팔 티, 아우터 없음, 운동화, 카고바지"
1335,14,2024-04-25,6.0,"기타 모자, 나일론 팬츠, 반팔 티, 스니커즈/캔버스, 집업"
1336,14,2024-04-26,6.0,"구두/로퍼, 반바지, 반팔 티, 셔츠/블라우스, 아우터 없음, 장목양말"


In [25]:
# 평균기온(°C) column의 각 class에 포함된 개수가 10개 이하라면 제외
df_limit = df_limit.copy()
df_limit = df_limit.groupby('평균기온(°C)').filter(lambda x: len(x) > 10)
df_limit['평균기온(°C)'].unique()

/tmp/ipykernel_251565/1967549491.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_limit = df_limit.groupby('평균기온(°C)').filter(lambda x: len(x) > 10)


[5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0]
Categories (10, float64): [0.0 < 1.0 < 2.0 < 3.0 ... 6.0 < 7.0 < 8.0 < 9.0]

In [26]:
# 각 user의 온도 별 예제 개수
df_limit.groupby(['userId', '평균기온(°C)']).size()

/tmp/ipykernel_251565/3554806506.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_limit.groupby(['userId', '평균기온(°C)']).size()


userId  평균기온(°C)
1       0.0          0
        1.0          2
        2.0          0
        3.0         14
        4.0         21
                    ..
14      5.0         25
        6.0         37
        7.0         35
        8.0         39
        9.0         11
Length: 140, dtype: int64

In [27]:
# '평균기온(°C)'의 각 범주를 고려하여 데이터를 분할
train_data = []
val_data = []
test_data = []
# 각 user별로 온도 범주의 데이터가 적은 경우 기록
user_category_not_valid = {}

for user in df_limit['userId'].unique():
    for category in df_limit['평균기온(°C)'].unique():
        category_data = df_limit[(df_limit['평균기온(°C)'] == category) & (df_limit['userId'] == user)]
        
        if category_data.shape[0] < 20:
            if user not in user_category_not_valid:
                user_category_not_valid[user] = [category]
            else:
                user_category_not_valid[user].append(category)
            train_data.append(category_data)
            continue

        # 먼저 전체 데이터의 50%를 훈련 데이터로 분할
        train, temp = train_test_split(category_data, test_size=0.5, random_state=42)
        
        # 남은 데이터를 반으로 나누어 검증 데이터와 테스트 데이터로 분할
        val, test = train_test_split(temp, test_size=0.3, random_state=42)
        
        train_data.append(train)
        val_data.append(val)
        test_data.append(test)

print(user_category_not_valid)
# 각 데이터 세트를 하나의 DataFrame으로 병합
train_data_df = pd.concat(train_data)
val_data_df = pd.concat(val_data)
test_data_df = pd.concat(test_data)

# 평균기온 column을 범주형으로 변경
train_data_df['평균기온(°C)'] = train_data_df['평균기온(°C)'].astype('float64')
val_data_df['평균기온(°C)'] = val_data_df['평균기온(°C)'].astype('float64')
test_data_df['평균기온(°C)'] = test_data_df['평균기온(°C)'].astype('float64')

{1: [3.0, 1.0, 7.0, 8.0, 9.0, 2.0], 2: [5.0, 4.0, 3.0, 1.0, 7.0, 9.0, 2.0], 3: [5.0, 1.0, 9.0, 2.0], 4: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0], 5: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0], 6: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0], 7: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0], 8: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 9.0, 2.0], 9: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0], 10: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0], 11: [4.0, 3.0, 1.0, 9.0, 2.0], 12: [5.0, 6.0, 1.0, 7.0, 8.0, 9.0, 2.0], 13: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0], 14: [1.0, 9.0, 2.0]}


In [28]:
train_data_df['옷 조합'].nunique(), val_data_df['옷 조합'].nunique(), test_data_df['옷 조합'].nunique()

(570, 167, 103)

In [29]:
# train_data_df와 val_data_df의 옷 조합 column의 고유한 값이 같은 경우의 개수
val_data_df['옷 조합'].unique()
train_data_df['옷 조합'].unique()
set(train_data_df['옷 조합'].unique()) & set(val_data_df['옷 조합'].unique())

{'가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음, 재킷',
 '가디건, 구두/로퍼, 데님팬츠, 반팔 티, 액세서리 없음',
 '가디건, 기타 모자, 긴팔 티, 데님팬츠, 스니커즈/캔버스',
 '가디건, 데님팬츠, 반팔 티, 셔츠/블라우스, 스니커즈/캔버스, 액세서리 없음',
 '가디건, 데님팬츠, 반팔 티, 스니커즈/캔버스, 액세서리 없음',
 '가디건, 면바지, 반팔 티, 셔츠/블라우스, 스니커즈/캔버스, 액세서리 없음',
 '구두/로퍼, 기타 모자, 긴팔 티, 데님팬츠, 재킷',
 '구두/로퍼, 기타 모자, 긴팔 티, 데님팬츠, 집업',
 '구두/로퍼, 기타 모자, 데님팬츠, 반팔 티, 셔츠/블라우스, 아우터 없음',
 '구두/로퍼, 기타 모자, 데님팬츠, 반팔 티, 아우터 없음',
 '구두/로퍼, 기타 모자, 맨투맨, 반바지, 아우터 없음, 장목양말',
 '구두/로퍼, 니트, 데님팬츠, 반팔 티, 아우터 없음, 액세서리 없음',
 '구두/로퍼, 니트, 데님팬츠, 반팔 티, 액세서리 없음, 코트',
 '구두/로퍼, 니트, 면바지, 반팔 티, 액세서리 없음, 코트',
 '구두/로퍼, 데님팬츠, 반팔 티, 셔츠/블라우스, 아우터 없음, 액세서리 없음',
 '구두/로퍼, 데님팬츠, 반팔 티, 셔츠/블라우스, 액세서리 없음, 재킷',
 '구두/로퍼, 데님팬츠, 반팔 티, 아우터 없음, 액세서리 없음',
 '구두/로퍼, 데님팬츠, 반팔 티, 액세서리 없음, 재킷',
 '구두/로퍼, 맨투맨, 면바지, 반팔 티, 아우터 없음, 액세서리 없음',
 '구두/로퍼, 면바지, 반팔 티, 셔츠/블라우스, 아우터 없음, 액세서리 없음',
 '구두/로퍼, 면바지, 반팔 티, 셔츠/블라우스, 액세서리 없음, 재킷',
 '구두/로퍼, 면바지, 반팔 티, 셔츠/블라우스, 액세서리 없음, 집업',
 '구두/로퍼, 면바지, 반팔 티, 아우터 없음, 액세서리 없음',
 '구두/로퍼, 반바지, 반팔 셔츠/블라우스, 반팔 티, 아우터 없음, 장목양말',
 '구두/로퍼, 반바지, 반팔 

In [30]:
'''# 중복된 행 찾기
duplicated_rows = train_data_df[train_data_df.duplicated(['옷 조합'], keep=False)]
duplicated_rows.sort_values(by='옷 조합')'''

"# 중복된 행 찾기\nduplicated_rows = train_data_df[train_data_df.duplicated(['옷 조합'], keep=False)]\nduplicated_rows.sort_values(by='옷 조합')"

In [31]:
# user-item-temperature의 3-dimensional tensor를 생성

# 사용자, 아이템, 온도 범주별로 그룹화하고 각 그룹의 크기를 계산
grouped = train_data_df.groupby(['userId', '옷 조합', '평균기온(°C)'], observed=True).size().reset_index(name='count')
grouped[(grouped['userId'] == 14) & (grouped['count'] > 1)]

,userId,옷 조합,평균기온(°C),count
790,14,"구두/로퍼, 기타 모자, 긴팔 티, 데님팬츠, 코트",2.0,2
816,14,"기타 모자, 긴팔 티, 데님팬츠, 스니커즈/캔버스, 재킷",3.0,2
834,14,"기타 모자, 나일론 팬츠, 반팔 티, 스니커즈/캔버스, 아우터 없음",9.0,2
844,14,"기타 모자, 데님팬츠, 바람막이, 반팔 티, 운동화",4.0,2
850,14,"기타 모자, 데님팬츠, 반팔 티, 스니커즈/캔버스, 아우터 없음, 후드티",3.0,2
863,14,"기타 모자, 면바지, 반팔 티, 스니커즈/캔버스, 아우터 없음",7.0,2
875,14,"기타 모자, 반바지, 반팔 티, 스니커즈/캔버스, 아우터 없음, 장목양말",7.0,2
876,14,"기타 모자, 반바지, 반팔 티, 스니커즈/캔버스, 아우터 없음, 장목양말",8.0,2
884,14,"기타 모자, 반팔 티, 스니커즈/캔버스, 아우터 없음, 카고바지",9.0,2
887,14,"기타 모자, 반팔 티, 아우터 없음, 운동화, 카고바지",5.0,2


In [32]:
'''# 3차원 텐서 생성
tensor = np.zeros((train_data_df['userId'].nunique(), train_data_df['옷 조합'].nunique(), train_data_df['평균기온(°C)'].nunique()))

for _, row in grouped.iterrows():
    user_idx = train_data_df['userId'].unique().tolist().index(row['userId'])
    item_idx = train_data_df['옷 조합'].unique().tolist().index(row['옷 조합'])
    temp_idx = train_data_df['평균기온(°C)'].unique().tolist().index(row['평균기온(°C)'])
    tensor[user_idx, item_idx, temp_idx] = row['count']'''

"# 3차원 텐서 생성\ntensor = np.zeros((train_data_df['userId'].nunique(), train_data_df['옷 조합'].nunique(), train_data_df['평균기온(°C)'].nunique()))\n\nfor _, row in grouped.iterrows():\n    user_idx = train_data_df['userId'].unique().tolist().index(row['userId'])\n    item_idx = train_data_df['옷 조합'].unique().tolist().index(row['옷 조합'])\n    temp_idx = train_data_df['평균기온(°C)'].unique().tolist().index(row['평균기온(°C)'])\n    tensor[user_idx, item_idx, temp_idx] = row['count']"

In [33]:
'''# 고유한 값들의 리스트를 미리 생성
user_unique_list = sorted(train_data_df['userId'].unique().tolist())
item_unique_list = sorted(train_data_df['옷 조합'].unique().tolist())
temp_unique_list = sorted(train_data_df['평균기온(°C)'].unique().tolist())

# 3차원 텐서 생성
tensor = np.zeros((len(user_unique_list), len(item_unique_list), len(temp_unique_list)))

for _, row in grouped.iterrows():
    user_idx = user_unique_list.index(row['userId'])
    item_idx = item_unique_list.index(row['옷 조합'])
    temp_idx = temp_unique_list.index(row['평균기온(°C)'])
    tensor[user_idx, item_idx, temp_idx] = row['count']'''

"# 고유한 값들의 리스트를 미리 생성\nuser_unique_list = sorted(train_data_df['userId'].unique().tolist())\nitem_unique_list = sorted(train_data_df['옷 조합'].unique().tolist())\ntemp_unique_list = sorted(train_data_df['평균기온(°C)'].unique().tolist())\n\n# 3차원 텐서 생성\ntensor = np.zeros((len(user_unique_list), len(item_unique_list), len(temp_unique_list)))\n\nfor _, row in grouped.iterrows():\n    user_idx = user_unique_list.index(row['userId'])\n    item_idx = item_unique_list.index(row['옷 조합'])\n    temp_idx = temp_unique_list.index(row['평균기온(°C)'])\n    tensor[user_idx, item_idx, temp_idx] = row['count']"

In [34]:
# pivot_table을 이용한 user-item matrix 생성
train_data_df_value = train_data_df.copy()
train_data_df_value['평균기온(°C)'] = train_data_df_value['평균기온(°C)'].astype('float32')
UI_temp = train_data_df_value.pivot_table(index='userId', columns='옷 조합', values='평균기온(°C)', fill_value=0)

In [35]:
UI_val = UI_temp.copy()
# UI_val의 값을 모두 0으로 초기화
UI_val = UI_val.map(lambda x: 0.0)
for user in UI_temp.index:
    for item in UI_temp.columns:
        # validation에 해당 user-item이 있는 경우 해당 user-item의 평균을 기록
        if item in val_data_df[val_data_df['userId'] == user]['옷 조합'].values:
            UI_val.loc[user, item] = val_data_df[(val_data_df['userId'] == user) & (val_data_df['옷 조합'] == item)]['평균기온(°C)'].mean()

In [36]:
UI_test = UI_temp.copy()
# UI_val의 값을 모두 0으로 초기화
UI_test = UI_test.map(lambda x: 0.0)
for user in UI_temp.index:
    for item in UI_temp.columns:
        # validation에 해당 user-item이 있는 경우 해당 user-item의 평균을 기록
        if item in test_data_df[test_data_df['userId'] == user]['옷 조합'].values:
            UI_test.loc[user, item] = test_data_df[(test_data_df['userId'] == user) & (test_data_df['옷 조합'] == item)]['평균기온(°C)'].mean()

In [37]:
# UI_val, UI_test의 값이 0이 아닌 경우의 개수
UI_val[UI_val != 0].count().sum(), UI_test[UI_test != 0].count().sum()

(112, 64)

In [38]:
# pivot_table을 이용한 user_
UI_count = train_data_df.pivot_table( index='userId', columns='옷 조합', aggfunc='size', fill_value=0)
# 해당 user의 총 예제 개수로 각각의 row를 나눔
UI_count_div = UI_count.div(UI_count.sum(axis=1), axis=0)

In [39]:
# user-item matrix에 기록된 값이 존재하는 경우 1, 아닌 경우 0으로 변환하여 R_df에 기록
R_df = UI_temp.map(lambda x: 1 if x != 0 else 0)
R_np = np.array(R_df)
R_np.sum(axis=0)

array([1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 9, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1,
       3, 1, 1, 2, 1, 1, 1, 2, 3, 4, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 2, 2, 1, 1, 1, 3, 3, 5, 1, 5, 1, 2, 1, 1, 2, 3, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 5, 3, 1, 2, 2, 1,
       1, 1, 2, 4, 2, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 6, 3, 3, 2, 1, 1, 2, 1, 5,
       1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3,
       1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1,

In [40]:
'''# tensor에 기록된 값이 존재하는 경우 1, 존재하지 않는 경우 0
R_np = np.where(tensor > 0, 1, 0)
sum_along_all_other_axes = R_np.sum(axis=tuple(range(1, R_np.ndim)))
sum_along_all_other_axes'''

'# tensor에 기록된 값이 존재하는 경우 1, 존재하지 않는 경우 0\nR_np = np.where(tensor > 0, 1, 0)\nsum_along_all_other_axes = R_np.sum(axis=tuple(range(1, R_np.ndim)))\nsum_along_all_other_axes'

In [41]:
# 각 열의 합이 2 이상(여러 유저가 해당 옷 조합을 선택한 경우)인 열을 찾음
columns_with_sum_over_2 = R_df.columns[R_df.sum() >= 2]

In [42]:
'''UI_temp[columns_with_sum_over_2]'''

'UI_temp[columns_with_sum_over_2]'

In [43]:
'''columns_with_sum_over_2'''

'columns_with_sum_over_2'

In [44]:
'''# 해당 조합의 인덱스
column_index = []
for i in columns_with_sum_over_2:
    column_index.append(R_df.columns.get_loc(i))
    column_index'''

'# 해당 조합의 인덱스\ncolumn_index = []\nfor i in columns_with_sum_over_2:\n    column_index.append(R_df.columns.get_loc(i))\n    column_index'

In [45]:
'''for row in R_np:
    print(row)'''

'for row in R_np:\n    print(row)'

In [46]:
# CF를 위한 초기값 설정
Y = np.array(UI_temp) 
Y = Y.T
count = np.array(UI_count_div)
count = count.T
print(Y.shape)
R = Y != 0 
n_u = Y.shape[1]
n_o = Y.shape[0]

(570, 14)


In [47]:
# validation, test
Y_val = np.array(UI_val)
Y_val = Y_val.T
Y_test = np.array(UI_test)
Y_test = Y_test.T

In [48]:
# 기록이 존재하는 값의 평균을 구함
o_sum = Y.sum(axis=1)
o_count = R.sum(axis=1)
o_mean = o_sum / o_count
o_mean = o_mean.reshape(-1, 1)

In [49]:
'''Y[column_index]'''

'Y[column_index]'

In [50]:
Y_stand = Y - (o_mean * R)
Y_val_stand = Y_val - (o_mean * (Y_val != 0))
Y_test_stand = Y_test - (o_mean * (Y_test != 0))

In [51]:
def cofi_cost_func_v(O, U, b, Y, R, lambda_):
    j = (tf.linalg.matmul(O, tf.transpose(U)) + b - Y )*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(O**2) + tf.reduce_sum(U**2))
    return J

In [52]:
# user, outfit의 수
n_o, n_u = Y.shape
# latent factor의 수
num_features = 30

# (U,O)를 초기화하고 tf.Variable로 등록하여 추적
tf.random.set_seed(1234) # for consistent results
U = tf.Variable(tf.random.normal((n_u,  num_features),dtype=tf.float64),  name='U')
O = tf.Variable(tf.random.normal((n_o, num_features),dtype=tf.float64),  name='O')
b = tf.Variable(tf.random.normal((1,          n_u),   dtype=tf.float64),  name='b')

# optimizer 초기화
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

2024-05-09 20:23:08.801071: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 20:23:09.286270: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 20:23:09.286340: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 20:23:09.289988: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 20:23:09.290096: I external/local_xla/xla/stream_executor

In [53]:
J = cofi_cost_func_v(O, U, b, Y_stand, R, 1.5)

2024-05-09 20:23:11.131288: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [54]:
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 27376.00


In [55]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # TensorFlow의 GradientTape 사용
    # 연산을 기록하여 cost에 대한 gradient를 자동으로 계산
    with tf.GradientTape() as tape:

        # cost 계산 (forward pass included in cost)
        cost_value = cofi_cost_func_v(O, U, b, Y_stand, R, lambda_)

    # GradientTape를 통해 자동 미분
    # loss에 대한 trainable parameter의 gradient를 계산
    grads = tape.gradient( cost_value, [O,U,b] )

    # optimizer를 사용하여 trainable parameter를 업데이트
    optimizer.apply_gradients( zip(grads, [O,U,b]) )

    # Log periodically.
    if (iter + 1) % 20 == 0:
        print(f"Training loss at iteration {iter + 1}: {cost_value:0.1f}")
        print(f'validation loss at iteration {iter + 1}: {cofi_cost_func_v(O, U, b, Y_val_stand, R, lambda_):0.1f}')

2024-05-09 20:23:11.753078: I external/local_xla/xla/service/service.cc:168] XLA service 0xc116b90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-09 20:23:11.753131: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-05-09 20:23:11.781490: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-09 20:23:11.825944: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1715253791.940031  251565 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training loss at iteration 20: 1901.2
validation loss at iteration 20: 1932.5
Training loss at iteration 40: 381.5
validation loss at iteration 40: 539.1
Training loss at iteration 60: 121.3
validation loss at iteration 60: 299.3
Training loss at iteration 80: 71.7
validation loss at iteration 80: 257.0
Training loss at iteration 100: 60.6
validation loss at iteration 100: 246.2
Training loss at iteration 120: 57.2
validation loss at iteration 120: 242.4
Training loss at iteration 140: 55.8
validation loss at iteration 140: 240.4
Training loss at iteration 160: 55.1
validation loss at iteration 160: 239.2
Training loss at iteration 180: 54.7
validation loss at iteration 180: 238.4
Training loss at iteration 200: 54.4
validation loss at iteration 200: 237.9


In [56]:
# 훈련된 tf.Variable 파일로 저장
model_version = '1.0'
checkpoint_path = f'../model/{model_version}/'
os.makedirs(checkpoint_path, exist_ok=True)
checkpoint = tf.train.Checkpoint(optimizer=optimizer, O=O, U=U, b=b)
checkpoint.save(os.path.join(checkpoint_path, 'parameters.ckpt'))
UI_temp.to_csv(os.path.join(checkpoint_path, 'UI_temp.csv'))

# UI_temp및 UI_count_div를 저장

In [57]:
# 'real_temperature'는 실제 온도입니다.
real_temperature = 14.8

# 실제 온도를 범주화합니다.
categorized_temperature = pd.cut([real_temperature], bins=np.round(np.arange(min_temp -5, max_temp+5, 5), 1), labels=np.arange(0, (max_temp-min_temp)//5+2))


# 첫 번째 요소를 선택하여 범주화된 온도를 얻습니다.
categorized_temperature = categorized_temperature[0]
categorized_temperature

6.0

In [58]:
'''# 예측을 수행하기 위해 모든 user-item에 대한 예측값을 계산
p = np.matmul(O.numpy(), np.transpose(U.numpy())) + b.numpy()
# 실제 온도
# 평균을 적용하고 temp를 빼서 값이 작을수록 실제 온도에 가깝도록 함. 이 때 각 user-item의 사용 횟수를 가중하여 많이 사용한 item이 추천되도록 함
pm = np.power(p + o_mean - categorized_temperature, 2)  -count * 0.4

# 각 user마다 반복
for i in range(n_u) :
    my_predictions = pm[:,i]

    # sort predictions
    ix = tf.argsort(my_predictions, direction='ASCENDING')

    df_predict = UI_temp[UI_temp.columns[ix[0:10]]].copy()
    df_predict = df_predict.round(0)
    df_predict.to_csv(f'../data/predictions/male/user_{i+1}_predictions.csv')'''

"# 예측을 수행하기 위해 모든 user-item에 대한 예측값을 계산\np = np.matmul(O.numpy(), np.transpose(U.numpy())) + b.numpy()\n# 실제 온도\n# 평균을 적용하고 temp를 빼서 값이 작을수록 실제 온도에 가깝도록 함. 이 때 각 user-item의 사용 횟수를 가중하여 많이 사용한 item이 추천되도록 함\npm = np.power(p + o_mean - categorized_temperature, 2)  -count * 0.4\n\n# 각 user마다 반복\nfor i in range(n_u) :\n    my_predictions = pm[:,i]\n\n    # sort predictions\n    ix = tf.argsort(my_predictions, direction='ASCENDING')\n\n    df_predict = UI_temp[UI_temp.columns[ix[0:10]]].copy()\n    df_predict = df_predict.round(0)\n    df_predict.to_csv(f'../data/predictions/male/user_{i+1}_predictions.csv')"

In [59]:
user_category_not_valid

{1: [3.0, 1.0, 7.0, 8.0, 9.0, 2.0],
 2: [5.0, 4.0, 3.0, 1.0, 7.0, 9.0, 2.0],
 3: [5.0, 1.0, 9.0, 2.0],
 4: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0],
 5: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0],
 6: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0],
 7: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0],
 8: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 9.0, 2.0],
 9: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0],
 10: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0],
 11: [4.0, 3.0, 1.0, 9.0, 2.0],
 12: [5.0, 6.0, 1.0, 7.0, 8.0, 9.0, 2.0],
 13: [5.0, 6.0, 4.0, 3.0, 1.0, 7.0, 8.0, 9.0, 2.0],
 14: [1.0, 9.0, 2.0]}

In [60]:
# 예측을 수행하기 위해 모든 user-item에 대한 예측값을 계산
p = np.matmul(O.numpy(), np.transpose(U.numpy())) + b.numpy()
# user_category_not_valid에 해당하지 않는 경우에 대해 precision, recall, f1_score 계산
# 평균을 위한 초기화
precision_m, recall_m, f1_score_m, count_m = 0, 0, 0, 0
for i in range(n_u):
    for category in df_limit['평균기온(°C)'].unique():
        
        # 실제 온도
        # 평균을 적용하고 temp를 빼서 값이 작을수록 실제 온도에 가깝도록 함. 이 때 각 user-item의 사용 횟수를 가중하여 많이 사용한 item이 추천되도록 함
        pm = np.power(p + o_mean - category, 2)  -count * 28
        my_predictions = pm[:,i]

        # sort predictions
        ix = tf.argsort(my_predictions, direction='ASCENDING')

        df_predict = UI_temp[UI_temp.columns[ix[0:3]]].copy()
        # df_predict의 columns와 test_data_df의 '옷 조합' column을 비교하여 일치하는 경우의 개수를 계산
        predict = df_predict.columns.astype(str)
        
        # user i에 대한 예측을 파일로 저장
        os.makedirs(f'../data/predictions/male/user_{i+1}', exist_ok=True)
        # Save predictions to file in user's directory
        with open(f'../data/predictions/male/user_{i+1}/predictions_{category}.txt', 'w') as f:
            for item in predict:
                f.write("%s\n" % item)
        
        if i+1 in user_category_not_valid and category in user_category_not_valid[i+1]:
            print(f'{i+1}번 user, {category}도 데이터가 부족하여 제외합니다.')
            continue
        
        count_m += 1
        
        label = test_data_df[(test_data_df['userId'] == i+1) & (test_data_df['평균기온(°C)'] == category)]['옷 조합'].astype(str)
        
        precision = len(set(predict) & set(label)) / len(set(predict))
        print(f'{i+1}번 user, {category}도 prediction: {predict}')
        print(f'{i+1}번 user, {category}도 label: {label}')
        recall = len(set(predict) & set(label)) / len(set(label))
        if precision + recall == 0:
            '''print(f'0인 경우')
            print(f'{i+1}번 user, {category}도 예측 결과: {predict}, 실제 결과: {label} ')'''  
            f1_score = 0
        else:
            '''print(f'0이 아닌 경우')
            print(f'{i+1}번 user, {category}도 예측 결과: {predict}, 실제 결과: {label} ')'''  
            f1_score = 2 * (precision * recall) / (precision + recall)
        precision_m += precision
        recall_m += recall
        f1_score_m += f1_score
        print(precision, recall, f1_score)
precision_m /= count_m
recall_m /= count_m
f1_score_m /= count_m
print(f'평균 precision: {precision_m}, 평균 recall: {recall_m}, 평균 f1_score: {f1_score_m}')

1번 user, 5.0도 prediction: Index(['니트, 데님팬츠, 반팔 티, 스니커즈/캔버스, 액세서리 없음, 재킷',
       '가디건, 구두/로퍼, 데님팬츠, 반팔 티, 셔츠/블라우스, 액세서리 없음',
       '면바지, 반팔 티, 비니, 스니커즈/캔버스, 재킷'],
      dtype='object', name='옷 조합')
1번 user, 5.0도 label: 81     데님팬츠, 반팔 티, 셔츠/블라우스, 스니커즈/캔버스, 액세서리 없음, 조끼
68                 면바지, 아우터 없음, 액세서리 없음, 운동화, 후드티
71      데님팬츠, 반팔 티, 셔츠/블라우스, 아우터 없음, 액세서리 없음, 운동화
104     면바지, 반팔 티, 셔츠/블라우스, 스니커즈/캔버스, 액세서리 없음, 코트
Name: 옷 조합, dtype: object
0.0 0.0 0
1번 user, 6.0도 prediction: Index(['긴팔 티, 데님팬츠, 스니커즈/캔버스, 액세서리 없음, 재킷', '구두/로퍼, 데님팬츠, 반팔 티, 액세서리 없음, 재킷',
       '가디건, 긴팔 티, 면바지, 운동화, 장목양말'],
      dtype='object', name='옷 조합')
1번 user, 6.0도 label: 41                      구두/로퍼, 기타 모자, 니트, 면바지, 반팔 티, 재킷
1                             기타 모자, 반바지, 반팔 티, 운동화, 재킷
86    기타 모자, 데님팬츠, 민소매 티, 민소매 티, 반팔 티, 셔츠/블라우스, 스니커즈...
76                         긴팔 티, 데님팬츠, 액세서리 없음, 운동화, 재킷
Name: 옷 조합, dtype: object
0.0 0.0 0
1번 user, 4.0도 prediction: Index(['기타 모자, 긴팔 티, 나일론 팬츠, 운동화, 재킷, 집업',
       '데님팬츠, 반팔 티

In [70]:
CF_pred = p + o_mean
CF_pred_df = pd.DataFrame(CF_pred.T, columns=UI_temp.columns, index=UI_temp.index)
CF_pred_df

옷 조합,"가디건, 구두/로퍼, 기타 모자, 긴팔 티, 장목양말, 트레이닝/조거 팬츠","가디건, 구두/로퍼, 기타 모자, 긴팔 티, 코트, 트레이닝/조거 팬츠","가디건, 구두/로퍼, 긴팔 티, 데님팬츠, 액세서리 없음","가디건, 구두/로퍼, 긴팔 티, 데님팬츠, 액세서리 없음, 코트","가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음","가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음, 재킷","가디건, 구두/로퍼, 니트, 데님팬츠, 머플러, 반팔 티, 코트","가디건, 구두/로퍼, 니트, 데님팬츠, 액세서리 없음","가디건, 구두/로퍼, 데님팬츠, 반팔 티, 셔츠/블라우스, 액세서리 없음","가디건, 구두/로퍼, 데님팬츠, 반팔 티, 액세서리 없음",...,"비니, 셔츠/블라우스, 아우터 없음, 운동화, 트레이닝/조거 팬츠","비니, 셔츠/블라우스, 운동화, 집업, 트레이닝/조거 팬츠","셔츠/블라우스, 스니커즈/캔버스, 슬랙스, 액세서리 없음, 재킷","셔츠/블라우스, 스니커즈/캔버스, 액세서리 없음, 트레이닝/조거 팬츠, 패딩","셔츠/블라우스, 슬랙스, 액세서리 없음, 운동화, 재킷","스니커즈/캔버스, 아우터 없음, 액세서리 없음, 트레이닝/조거 팬츠, 후드티","스니커즈/캔버스, 액세서리 없음, 재킷, 카고바지, 후드티","스니커즈/캔버스, 액세서리 없음, 재킷, 트레이닝/조거 팬츠, 후드티","아우터 없음, 액세서리 없음, 운동화, 카고바지, 후드티","액세서리 없음, 운동화, 카고바지, 패딩, 후드티"
userId,,,,,,,,,,,,,,,,,,,,,
1,3.994414,4.994593,4.973695,2.959676,3.365331,2.959734,0.973817,2.959769,4.993703,4.975850,...,7.964578,3.973899,4.928560,0.942947,3.928745,1.973720,2.993685,3.973496,3.970256,0.943095
2,4.027159,5.027168,5.033271,3.033460,3.111713,3.033390,1.073793,3.033386,5.035807,5.007924,...,8.042714,4.012104,5.059318,1.050251,4.060961,2.033229,3.035747,4.033547,4.040762,1.050319
3,4.070774,5.070808,5.118390,3.130234,3.985123,3.130293,1.013413,3.130260,5.092960,5.114624,...,8.091600,4.122645,5.086039,1.092922,4.085062,2.118438,3.093034,4.117564,4.094582,1.092955
4,3.847041,4.846938,4.847388,2.884135,3.861133,2.884099,0.849873,2.884075,4.841075,4.843018,...,7.848576,3.842292,4.955927,0.850744,3.955603,1.847353,2.840919,3.847491,3.849952,0.850840
5,4.083654,5.083646,5.091611,3.086024,3.510138,3.086064,1.083097,3.086057,5.084172,5.083793,...,8.086293,4.083572,5.092737,1.075675,4.092907,2.091580,3.084141,4.091612,4.085556,1.075720
6,3.889345,4.889322,4.879757,2.877855,3.644550,2.877850,0.884241,2.877826,4.884039,4.885150,...,7.879920,3.884649,4.871248,0.882671,3.871134,1.879799,2.884072,3.879744,3.881479,0.882604
7,3.918561,4.918545,4.888287,2.897505,3.345928,2.897214,0.917197,2.897525,4.906684,4.904488,...,7.901112,3.902370,4.895905,0.877678,3.896382,1.888214,2.906690,3.888411,3.905139,0.877639
8,3.982799,4.982678,4.948013,2.989988,3.907460,2.990187,1.007333,2.989954,4.981176,4.985154,...,7.996039,3.980576,5.015087,0.999867,4.015039,1.948115,2.981064,3.947939,3.986117,0.999813
9,3.902085,4.902078,4.940468,2.953880,4.040768,2.953876,0.944997,2.953849,4.928540,4.936433,...,7.945958,3.926901,4.958198,0.988313,3.957823,1.940474,2.928554,3.940601,3.939910,0.988304


In [80]:
train_data_df

,userId,작성일,평균기온(°C),옷 조합
36,1,2023-12-08,5.0,"긴팔 티, 액세서리 없음, 운동화, 재킷, 카고바지"
43,1,2023-10-20,5.0,"가디건, 구두/로퍼, 데님팬츠, 반팔 티, 셔츠/블라우스, 액세서리 없음"
98,1,2023-03-14,5.0,"면바지, 반팔 티, 비니, 스니커즈/캔버스, 재킷"
92,1,2023-03-24,5.0,"가디건, 데님팬츠, 반팔 티, 액세서리 없음, 운동화"
103,1,2023-03-08,5.0,"니트, 데님팬츠, 반팔 티, 스니커즈/캔버스, 액세서리 없음, 재킷"
...,...,...,...,...
1108,14,2023-01-03,2.0,"긴팔 티, 면바지, 스니커즈/캔버스, 액세서리 없음, 재킷"
1112,14,2023-01-18,2.0,"기타 모자, 데님팬츠, 반팔 티, 스니커즈/캔버스, 아우터 없음, 후드티"
1240,14,2023-11-30,2.0,"구두/로퍼, 기타 모자, 긴팔 티, 데님팬츠, 코트"
1241,14,2023-12-01,2.0,"구두/로퍼, 기타 모자, 긴팔 티, 데님팬츠, 코트"


In [82]:
# df_combtest와 CF_pred_df의 각 userId와 옷 조합이 일치하는 경우 CF_pred_df의 값을 df_combtest에 'prediction'이라는 열을 만들어 저장
df_comb_train = train_data_df.copy()
df_comb_train['prediction'] = 0
for i in range(n_u):
    for combination in CF_pred_df.columns:
        if combination in df_comb_train[df_comb_train['userId'] == i+1]['옷 조합'].values :
            df_comb_train.loc[(df_comb_train['userId'] == i+1) & (df_pred_comb['옷 조합'] == combination), 'prediction'] = CF_pred_df.loc[i+1, combination]
df_comb_train

/tmp/ipykernel_251565/4061284344.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.9937032349513775' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_comb_train.loc[(df_comb_train['userId'] == i+1) & (df_pred_comb['옷 조합'] == combination), 'prediction'] = CF_pred_df.loc[i+1, combination]


,userId,작성일,평균기온(°C),옷 조합,prediction
36,1,2023-12-08,5.0,"긴팔 티, 액세서리 없음, 운동화, 재킷, 카고바지",4.847966
43,1,2023-10-20,5.0,"가디건, 구두/로퍼, 데님팬츠, 반팔 티, 셔츠/블라우스, 액세서리 없음",4.993703
98,1,2023-03-14,5.0,"면바지, 반팔 티, 비니, 스니커즈/캔버스, 재킷",4.993685
92,1,2023-03-24,5.0,"가디건, 데님팬츠, 반팔 티, 액세서리 없음, 운동화",4.993657
103,1,2023-03-08,5.0,"니트, 데님팬츠, 반팔 티, 스니커즈/캔버스, 액세서리 없음, 재킷",4.719229
...,...,...,...,...,...
1108,14,2023-01-03,2.0,"긴팔 티, 면바지, 스니커즈/캔버스, 액세서리 없음, 재킷",1.981231
1112,14,2023-01-18,2.0,"기타 모자, 데님팬츠, 반팔 티, 스니커즈/캔버스, 아우터 없음, 후드티",3.128258
1240,14,2023-11-30,2.0,"구두/로퍼, 기타 모자, 긴팔 티, 데님팬츠, 코트",3.231256
1241,14,2023-12-01,2.0,"구두/로퍼, 기타 모자, 긴팔 티, 데님팬츠, 코트",3.231256


In [83]:
# df_combtest와 CF_pred_df의 각 userId와 옷 조합이 일치하는 경우 CF_pred_df의 값을 df_combtest에 'prediction'이라는 열을 만들어 저장
df_comb_val = val_data_df.copy()
df_comb_val['prediction'] = 0
for i in range(n_u):
    for combination in CF_pred_df.columns:
        if combination in df_comb_val[df_comb_val['userId'] == i+1]['옷 조합'].values :
            df_comb_val.loc[(df_comb_val['userId'] == i+1) & (df_comb_val['옷 조합'] == combination), 'prediction'] = CF_pred_df.loc[i+1, combination]
df_comb_val

/tmp/ipykernel_251565/352405653.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.993443391830928' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_comb_val.loc[(df_comb_val['userId'] == i+1) & (df_comb_val['옷 조합'] == combination), 'prediction'] = CF_pred_df.loc[i+1, combination]


,userId,작성일,평균기온(°C),옷 조합,prediction
78,1,2023-04-13,5.0,"구두/로퍼, 데님팬츠, 반팔 티, 셔츠/블라우스, 액세서리 없음, 재킷",5.993443
0,1,2024-04-24,5.0,"구두/로퍼, 반바지, 반팔 티, 셔츠/블라우스, 액세서리 없음, 재킷",0.000000
91,1,2023-03-26,5.0,"니트, 데님팬츠, 반팔 티, 스니커즈/캔버스, 아우터 없음, 액세서리 없음",5.897997
89,1,2023-03-28,5.0,"가디건, 반팔 티, 셔츠/블라우스, 액세서리 없음, 운동화, 트레이닝/조거 팬츠",0.000000
80,1,2023-04-11,5.0,"맨투맨, 면바지, 반팔 티, 스니커즈/캔버스, 아우터 없음, 액세서리 없음",1.959691
...,...,...,...,...,...
1179,14,2023-07-10,8.0,"기타 모자, 데님팬츠, 반팔 티, 샌들/슬리퍼, 셔츠/블라우스, 아우터 없음",0.000000
1191,14,2023-07-26,8.0,"기타 모자, 반바지, 반팔 티, 스니커즈/캔버스, 아우터 없음, 장목양말",7.651161
1180,14,2023-07-12,8.0,"기타 모자, 데님팬츠, 반팔 티, 스니커즈/캔버스, 아우터 없음",7.063601
1182,14,2023-07-14,8.0,"구두/로퍼, 기타 모자, 데님팬츠, 반팔 티, 아우터 없음",7.501097


In [74]:
CF_pred_df.columns

Index(['가디건, 구두/로퍼, 기타 모자, 긴팔 티, 장목양말, 트레이닝/조거 팬츠',
       '가디건, 구두/로퍼, 기타 모자, 긴팔 티, 코트, 트레이닝/조거 팬츠',
       '가디건, 구두/로퍼, 긴팔 티, 데님팬츠, 액세서리 없음',
       '가디건, 구두/로퍼, 긴팔 티, 데님팬츠, 액세서리 없음, 코트', '가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음',
       '가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음, 재킷',
       '가디건, 구두/로퍼, 니트, 데님팬츠, 머플러, 반팔 티, 코트', '가디건, 구두/로퍼, 니트, 데님팬츠, 액세서리 없음',
       '가디건, 구두/로퍼, 데님팬츠, 반팔 티, 셔츠/블라우스, 액세서리 없음',
       '가디건, 구두/로퍼, 데님팬츠, 반팔 티, 액세서리 없음',
       ...
       '비니, 셔츠/블라우스, 아우터 없음, 운동화, 트레이닝/조거 팬츠',
       '비니, 셔츠/블라우스, 운동화, 집업, 트레이닝/조거 팬츠',
       '셔츠/블라우스, 스니커즈/캔버스, 슬랙스, 액세서리 없음, 재킷',
       '셔츠/블라우스, 스니커즈/캔버스, 액세서리 없음, 트레이닝/조거 팬츠, 패딩',
       '셔츠/블라우스, 슬랙스, 액세서리 없음, 운동화, 재킷',
       '스니커즈/캔버스, 아우터 없음, 액세서리 없음, 트레이닝/조거 팬츠, 후드티',
       '스니커즈/캔버스, 액세서리 없음, 재킷, 카고바지, 후드티',
       '스니커즈/캔버스, 액세서리 없음, 재킷, 트레이닝/조거 팬츠, 후드티',
       '아우터 없음, 액세서리 없음, 운동화, 카고바지, 후드티', '액세서리 없음, 운동화, 카고바지, 패딩, 후드티'],
      dtype='object', name='옷 조합', length=570)

In [ ]:
categorized_temperature

[2.0]
Categories (10, float64): [0.0 < 1.0 < 2.0 < 3.0 ... 6.0 < 7.0 < 8.0 < 9.0]

In [72]:
df_weather
df_side = df_weather.copy().drop(columns=['지점', '지점명', '평균기온(°C)'])
df_side['일시'] = pd.to_datetime(df_side['일시'])
df_side

,일시,최저기온(°C),최고기온(°C),강수 계속시간(hr),평균 풍속(m/s),평균 상대습도(%)
0,2022-08-01,25.4,32.4,14.83,2.4,82.4
1,2022-08-02,25.5,28.5,18.58,2.8,90.0
2,2022-08-03,25.2,30.0,13.25,2.8,90.1
3,2022-08-04,26.0,32.9,0.42,2.5,79.3
4,2022-08-05,27.0,32.7,0.50,2.6,76.6
...,...,...,...,...,...,...
634,2024-04-26,11.4,26.0,NaN,1.9,61.1
635,2024-04-27,11.5,26.2,NaN,1.9,56.4
636,2024-04-28,13.0,28.9,NaN,1.6,53.5
637,2024-04-29,16.9,26.1,NaN,3.0,48.0
